## Data Viewer

In [38]:
import importlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import sys
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import pandas as pd
import numpy as np
import scipy
from scipy import ndimage
from skimage import io, transform
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets,models,transforms       
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import time
import os
import copy
import torch.nn.functional as F
cudnn.benchmark = True
plt.ion()
from functools import partial
import uproot

#import utility_functions as utils

In [43]:
import TPCParser as io
importlib.reload(io)

path = '/home/kris/Documents/coding/WAWTPC/krisTPCReco/TPCReco-kris/build/resources/'
filename1 = 'out.root'

file = uproot.open(path+filename1)

# Define the branch name
branch_name = "myChargeArray[3][3][256][512]"

# Iterate through the specified branch
for array in uproot.iterate(file["TPCData/Event"][branch_name], library="ak"):
    print(array)




[{'myChargeArray[3][3][256][512]': [[[[0, ..., 0], ...], ...], ...]}, ...]


In [46]:
projections = io.parseChargeMaps(path+filename1)

In [47]:
import TPCParser as io
importlib.reload(io)
normMergedImage = io.getMergedImages(projections)

In [49]:
###############################################
# <> SETUP PLOT <> #
event = 0

fig, axis = plt.subplots()
im = axis.imshow(normMergedImage[event], origin='lower', aspect='auto')

###############################################
# <> SETUP PLOT <> #

projNames = ("U", "V", "W")
fig2, axes2 = plt.subplots(1,3, figsize=(28,10))
for strip in range(0,3):
    axis2 = axes2[strip] 
    data = projections[event][:,:,strip]
    im2 = axis2.imshow(data, origin='lower', aspect='auto')    
    axis2.set_xlabel("time bin")
    axis2.set_ylabel(projNames[strip]+" strip")
    divider = make_axes_locatable(axis2)
    cax = divider.append_axes("right", size="5%", pad=0.4)
    fig2.colorbar(im2, cax=cax)
    plt.subplots_adjust(bottom=0.15, left=0.05, right=0.95, wspace=0.3)

IndentationError: unexpected indent (3546116553.py, line 3)

## Training Dataset preparation

In [5]:
#print(len(normMergedImage))             # entries
#print(len(normMergedImage[0]))          # strips
#print(len(normMergedImage[0][0]))       # timebins
#print(len(normMergedImage[0][0][0]))    # channels

In [6]:
#data_transforms = {'train':transforms.Compose([transforms.ToTensor])}
import TPCParser as io
importlib.reload(io)
tensor_data = torch.from_numpy(np.array(normMergedImage))
tensor_data = tensor_data.float()

In [7]:
import TPCParser as io
importlib.reload(io)
data = io.padTensorData(tensor_data)

In [8]:
#print(tensor_data.shape)
#print(data.shape)
#entry_index = 500  # Index of the entry to visualize
#image_entry = data[entry_index]
#plt.imshow(image_entry)
#plt.axis('off')
#plt.show()

In [9]:
num_classes = 3
resnet_model = models.resnet50(pretrained=True)

# Freeze the early layers
for param in resnet_model.parameters():
    param.requires_grad = False


    

/home/kris/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kris/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


: 